Read Clean data

In [ ]:
import pandas as pd
import numpy as np

final_df = pd.read_excel('/content/final_dataframe.xlsx')


Add year column (used for joins/merges later on)

In [ ]:
final_df['Year'] = final_df['Incident Date'].dt.year
final_df

,Claim Number,Date Received,Incident Date,Airport Code,Airport Name,Airline Name,Claim Type,Claim Site,Item,Disposition,Close Amount,Year
0,2006010699026,2005-01-02,2005-12-06,ABQ,Albuquerque International Sunport Airport,America West,Property Damage,Checkpoint,"Clothing - Shoes, belts, accessories, etc.",Approved,395,2005
1,2006010698959,2005-01-02,2005-12-24,LAS,McCarran International,Mexicana,Passenger Property Loss,Checked Baggage,Other,Denied,0,2005
2,2006010698980,2005-01-02,2005-12-26,IAH,Houston - George Bush Intercontinental Airport,Continental Airlines,Passenger Property Loss,Checkpoint,Cameras - Digital,Denied,0,2005
3,2006010698987,2005-01-02,2005-12-23,ORD,Chicago O'Hare International Airport,American Airlines,Passenger Property Loss,Checked Baggage,Jewelry - Fine,Denied,0,2005
4,2006010699028,2005-01-02,2005-12-28,PIT,Pittsburgh International Airport,USAir,Passenger Property Loss,Checked Baggage,Cameras - Digital,Denied,0,2005
...,...,...,...,...,...,...,...,...,...,...,...,...
137447,2015010219514,2014-12-31,2014-12-10,CLT,Charlotte/Douglas International Airport,Southwest Airlines,Employee Loss (MPCECA),Checked Baggage,-,Deny,0,2014
137448,2015010219516,2014-12-31,2014-12-08,FLL,Ft. Lauderdale-Hollywood International,Southwest Airlines,Property Damage,Checked Baggage,Computer & Accessories,Approve in Full,800,2014
137449,2015010219535,2014-12-31,2014-12-23,GFK,Grand Forks - Mark Andrews International,Allegiant Air,Passenger Property Loss,Checked Baggage,Cosmetics & Grooming,Deny,0,2014
137450,2015010819728,2014-12-26,2014-10-10,HDQ,Headquarters,-,Passenger Property Loss,Other,-,Approve in Full,256,2014


Extract geospatial airport data

In [ ]:
!pip install dbfread

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import zipfile

with zipfile.ZipFile("/content/dot_54907_DS3.zip", "r") as zip_ref:
    zip_ref.extractall("airport_data")


In [ ]:
import os
from dbfread import DBF
airport_filenames = ['/content/airport_data/bts_Airports_20210311/airport_'+str(year)+'.zip' for year in range(2005,2015)]

airport_locations = pd.DataFrame()
for filename in airport_filenames:
    with zipfile.ZipFile(filename, "r") as zip_ref:
        zip_ref.extractall(filename.split('.')[0])

dbfs = ['/content/airport_data/bts_Airports_20210311/airport_2005/airport_2005/airports.dbf',
        '/content/AnyConv.com__airports2006.xlsx',
        '/content/airport_data/bts_Airports_20210311/airport_2007/airport_2007/airports.dbf',
        '/content/airport_data/bts_Airports_20210311/airport_2008/airport_2008/airports.dbf',
        '/content/airport_data/bts_Airports_20210311/airport_2009/airport_2009/airports.dbf',
        '/content/airport_data/bts_Airports_20210311/airport_2010/airport_2010/airports.dbf',
        '/content/airport_data/bts_Airports_20210311/airport_2011/airport_2011/airports.dbf',
        '/content/airport_data/bts_Airports_20210311/airport_2012/airport_2012/airports.dbf',
        '/content/airport_data/bts_Airports_20210311/airport_2013/airport_2013/airports.dbf',
        '/content/airport_data/bts_Airports_20210311/airport_2014/airports_2014/airports.dbf']

for filename in dbfs:
    df = pd.DataFrame()
    if '2006' not in filename:
        table = DBF(filename)
        df = pd.DataFrame(iter(table))
        locations = df[['LOCID', 'STATE_NAME', 'COUNTY_NAM', 'COUNTY_ST', 'LATITUDE', 'LONGITUDE']]
        locations = locations.rename(columns={'LOCID': 'Airport Code'})
        print(int(filename.split('/')[4].split('_')[1]) )
        locations['Year'] = [int(filename.split('/')[4].split('_')[1]) for x in range(locations.shape[0])]
        airport_locations = pd.concat([airport_locations, locations], ignore_index=True)
    else:
        df = pd.read_excel(filename)
        locations = df[['LOCID,C,4', 'STATE_NAME,C,20', 'COUNTY_NAM,C,21', 'COUNTY_ST,C,2', 'LATITUDE,N,19,10', 'LONGITUDE,N,19,10']]
        locations = locations.rename(columns={'LOCID,C,4': 'Airport Code', 'STATE_NAME,C,20': 'STATE_NAME', 'COUNTY_NAM,C,21':'COUNTY_NAM', \
                                              'COUNTY_ST,C,2':'COUNTY_ST', 'LATITUDE,N,19,10':'LATITUDE', 'LONGITUDE,N,19,10':'LONGITUDE'})
        print(2006)
        locations['Year'] = [2006 for x in range(locations.shape[0])]
        airport_locations = pd.concat([airport_locations, locations], ignore_index=True)


2005
2006
2007
2008
2009
2010
2011
2012
2013
2014


Merge clean data and location data based on airport code and year

In [ ]:
incidents_airport_locations = pd.merge(final_df, airport_locations, on=['Airport Code', 'Year'], how='inner') #or left?

In [ ]:
incidents_airport_locations

,Claim Number,Date Received,Incident Date,Airport Code,Airport Name,Airline Name,Claim Type,Claim Site,Item,Disposition,Close Amount,Year,STATE_NAME,COUNTY_NAM,COUNTY_ST,LATITUDE,LONGITUDE
0,2006010699026,2005-01-02,2005-12-06,ABQ,Albuquerque International Sunport Airport,America West,Property Damage,Checkpoint,"Clothing - Shoes, belts, accessories, etc.",Approved,395,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194
1,2005020475693,2005-01-19,2005-01-09,ABQ,Albuquerque International Sunport Airport,Delta Air Lines,Passenger Property Loss,Checked Baggage,Other,Denied,NaN,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194
2,2005021075827,2005-01-19,2005-01-09,ABQ,Albuquerque International Sunport Airport,Southwest Airlines,Property Damage,Checked Baggage,Luggage (all types including footlockers),Denied,0,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194
3,2005021676222,2005-01-27,2005-01-07,ABQ,Albuquerque International Sunport Airport,Delta Air Lines,Passenger Property Loss,Checked Baggage,Other,Denied,0,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194
4,2005030776999,2005-02-01,2005-01-05,ABQ,Albuquerque International Sunport Airport,Southwest Airlines,Passenger Property Loss,Checked Baggage,"Clothing - Shoes, belts, accessories, etc.",Approved,60,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134163,2014120819020,2014-11-26,2014-08-11,SBN,South Bend Regional,UAL,Property Damage,Checked Baggage,Cosmetics & Grooming,-,-,2014,INDIANA,ST JOSEPH,IN,41.708222,-86.317333
134164,2014121019094,2014-12-10,2014-11-28,ENV,Wendover Municipal Airport,Allegiant Air,Property Damage,Checked Baggage,Baggage/Cases/Purses,-,-,2014,UTAH,TOOELE,UT,40.718694,-114.030889
134165,2014122419398,2014-12-24,2014-12-03,ENV,Wendover Municipal Airport,Allegiant Air,Property Damage,Checked Baggage,Cosmetics & Grooming,-,-,2014,UTAH,TOOELE,UT,40.718694,-114.030889
134166,2014121619235,2014-12-16,2014-09-14,ITH,"Tompkins County, Ithaca",USAir,Property Damage,Checked Baggage,-,-,-,2014,NEW YORK,TOMPKINS,NY,42.491361,-76.458722


Extract GDP data

In [ ]:
with zipfile.ZipFile("/content/CAGDP1.zip", "r") as zip_ref:
    zip_ref.extractall("CAGDP1")


In [ ]:
import chardet
CAGDP1 = pd.DataFrame()
count = 0
for filename in os.listdir('/content/CAGDP1'):

    state = filename.split('_')[1]
    if len(state) ==2 :
        count +=1
        
        with open(os.path.join('/content/CAGDP1', filename), 'rb') as f:
            result = chardet.detect(f.read())
        print(state, result['encoding'])
        gdp = pd.read_csv(os.path.join('/content/CAGDP1', filename), encoding=result['encoding']) #, on_bad_lines='skip'
        gdp = gdp[['GeoName', 'Description','Unit','2005','2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014']]
        gdp['GeoName'] = [county_state.split(',')[0].upper() for county_state in gdp['GeoName'].astype('str')]
        gdp['State'] = [state for x in range(gdp.shape[0])]
        CAGDP1 = pd.concat([CAGDP1, gdp], ignore_index=True)
print(count)
CAGDP1

WA ascii
NM ISO-8859-1
MN ascii
MO ascii
TX ascii
MS ascii
GA ascii
HI ascii
AR ascii
AZ ascii
CA ascii
NJ ascii
VT ascii
IL ascii
MT ascii
CO ascii
KS ascii
WI ascii
NY ascii
WV ascii
MD ascii
AK ascii
NV ascii
UT ascii
RI ascii
LA ascii
IN ascii
IA ascii
AL ascii
OK ascii
MA ascii
NH ascii
CT ascii
ND ascii
OR ascii
DE ascii
ME ascii
VA ascii
DC ascii
FL ascii
NE ascii
US ascii
KY ascii
OH ascii
WY ascii
SD ascii
TN ascii
NC ascii
ID ascii
SC ascii
MI ascii
PA ascii
52


,GeoName,Description,Unit,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,State
0,WASHINGTON,Real GDP (thousands of chained 2012 dollars),Thousands of chained 2012 dollars,340427957.0,353263484.0,373180589.0,381289932.0,370136625.0,380989464.0,387745206.0,400530582.0,410873880.0,424467606.0,WA
1,WASHINGTON,Chain-type quantity indexes for real GDP,Quantity index,84.994,88.199,93.172,95.196,92.412,95.121,96.808,100.0,102.582,105.976,WA
2,WASHINGTON,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,293675976.0,314265242.0,342024216.0,354794080.0,351245043.0,365497753.0,379574857.0,400530582.0,419092381.0,440647989.0,WA
3,ADAMS,Real GDP (thousands of chained 2012 dollars),Thousands of chained 2012 dollars,664552.0,703825.0,850911.0,850245.0,819289.0,825275.0,800279.0,813458.0,845135.0,812003.0,WA
4,ADAMS,Chain-type quantity indexes for real GDP,Quantity index,81.695,86.523,104.604,104.522,100.717,101.453,98.38,100.0,103.894,99.821,WA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9890,YORK,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,15705199.0,15930234.0,17204306.0,18009914.0,17691069.0,18113252.0,18127040.0,18084843.0,18833013.0,19276259.0,PA
9891,NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PA
9892,NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PA
9893,NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PA


Find inconsistencies i.e. county names that refer to the same county but are slighly different in eacg dataset e.g. 'MC CRACKEN' and 'MCCRACKEN'

In [ ]:
geoname = list(CAGDP1['GeoName'].unique())
countynam = list(incidents_airport_locations['COUNTY_NAM'].unique())
'''for geo in geoname:
    if geo not in countynam:
        print(geo)'''

def find_max_substring(str_list1, str_list2):
    max_substrings = []
    count = 0
    for str1 in str_list1:
        
        if str1 not in str_list2:
            count += 1
            max_substring = ''
            sub = ''
            for str2 in str_list2:
                for i in range(len(str2)):
                    for j in range(i + 1, len(str2) + 1):
                        substring = str2[i:j]
                        if substring in str1 and len(substring) > len(max_substring):
                            max_substring = substring
                            sub = str2
            max_substrings.append((str1, max_substring, sub))
    print(count)
    return max_substrings

for  a in  find_max_substring( sorted(countynam), sorted(geoname)):
    print(a)


"""
('ALBEMARLE', 'ALBEMARLE + CHARLOTTESVILLE')
('ALEUTIAN ISLANDS', 'ALEUTIANS EAST BOROUGH')
('ANCHORAGE',  'ANCHORAGE MUNICIPALITY')
('BETHEL', 'BETHEL CENSUS AREA')
('BRISTOL BAY', 'BRISTOL BAY BOROUGH')
('CORDOVA-MCCARTHY', 'VALDEZ-CORDOVA CENSUS AREA')
('FAIRBANKS', 'FAIRBANKS NORTH STAR BOROUGH')
('FAIRBANKS NORTH STAR', 'FAIRBANKS NORTH STAR BOROUGH')
('KENAI-COOK INLET',  'KENAI PENINSULA BOROUGH')
('KETCHIKAN', 'KETCHIKAN GATEWAY BOROUGH')
('KETCHIKAN GATEWAY',  'KETCHIKAN GATEWAY BOROUGH')
('KODIAK', 'KODIAK ISLAND BOROUGH')
('KODIAK ISLAND',  'KODIAK ISLAND BOROUGH')
('MAUI',  'MAUI + KALAWAO')
('MC CRACKEN',  'MCCRACKEN')
('MC KEAN',  'MCKEAN')
('MC LEAN', 'MCLEAN')
('MC LENNAN',  'MCLENNAN')
('NEWPORT NEWS', 'NEWPORT NEWS (INDEPENDENT CITY)')
('NEWPORT NEWS', 'NEWPORT NEWS (INDEPENDENT CITY)')
('ST CLAIR',  'EAU CLAIRE')
('ST JOHNS',  'ST. JOHNS')
('ST JOSEPH',  'ST. JOSEPH')
('ST LAWRENCE',  'ST. LAWRENCE')
('ST LOUIS',  'ST. LOUIS')
('ST LOUIS CITY',  'ST. LOUIS (INDEPENDENT CITY)')
('VALDEZ-CHITNA-WHITIE', 'VALDEZ-CORDOVA CENSUS AREA')
('WRANGELL-PETERSBURG', 'WRANGELL-PETERSBURG CENSUS AREA')
"""

38
('--PUERTO RICO', 'RICO', 'HENRICO')
('-VIRGIN ISLANDS-', ' ISLAND', 'KODIAK ISLAND BOROUGH')
('ALBEMARLE', 'ALBEMARLE', 'ALBEMARLE + CHARLOTTESVILLE')
('ALEUTIAN ISLANDS', 'ALEUTIAN', 'ALEUTIANS EAST BOROUGH')
('AMERICAN SAMOA', 'AN SA', 'SAN SABA')
('ANCHORAGE', 'ANCHORAGE', 'ANCHORAGE MUNICIPALITY')
('BETHEL', 'BETHEL', 'BETHEL CENSUS AREA')
('BRISTOL BAY', 'BRISTOL BAY', 'BRISTOL BAY BOROUGH')
('CORDOVA-MCCARTHY', 'CORDOVA', 'VALDEZ-CORDOVA CENSUS AREA')
('FAIRBANKS', 'FAIRBANKS', 'FAIRBANKS NORTH STAR BOROUGH')
('FAIRBANKS NORTH STAR', 'FAIRBANKS NORTH STAR', 'FAIRBANKS NORTH STAR BOROUGH')
('GUAM', 'GUA', 'GUADALUPE')
('KENAI-COOK INLET', 'KENAI', 'KENAI PENINSULA BOROUGH')
('KETCHIKAN', 'KETCHIKAN', 'KETCHIKAN GATEWAY BOROUGH')
('KETCHIKAN GATEWAY', 'KETCHIKAN GATEWAY', 'KETCHIKAN GATEWAY BOROUGH')
('KOBUK', 'OBU', 'YALOBUSHA')
('KODIAK', 'KODIAK', 'KODIAK ISLAND BOROUGH')
('KODIAK ISLAND', 'KODIAK ISLAND', 'KODIAK ISLAND BOROUGH')
('MAUI', 'MAUI', 'MAUI + KALAWAO')
('MC CRAC

"\n('ALBEMARLE', 'ALBEMARLE + CHARLOTTESVILLE')\n('ALEUTIAN ISLANDS', 'ALEUTIANS EAST BOROUGH')\n('ANCHORAGE',  'ANCHORAGE MUNICIPALITY')\n('BETHEL', 'BETHEL CENSUS AREA')\n('BRISTOL BAY', 'BRISTOL BAY BOROUGH')\n('CORDOVA-MCCARTHY', 'VALDEZ-CORDOVA CENSUS AREA')\n('FAIRBANKS', 'FAIRBANKS NORTH STAR BOROUGH')\n('FAIRBANKS NORTH STAR', 'FAIRBANKS NORTH STAR BOROUGH')\n('KENAI-COOK INLET',  'KENAI PENINSULA BOROUGH')\n('KETCHIKAN', 'KETCHIKAN GATEWAY BOROUGH')\n('KETCHIKAN GATEWAY',  'KETCHIKAN GATEWAY BOROUGH')\n('KODIAK', 'KODIAK ISLAND BOROUGH')\n('KODIAK ISLAND',  'KODIAK ISLAND BOROUGH')\n('MAUI',  'MAUI + KALAWAO')\n('MC CRACKEN',  'MCCRACKEN')\n('MC KEAN',  'MCKEAN')\n('MC LEAN', 'MCLEAN')\n('MC LENNAN',  'MCLENNAN')\n('NEWPORT NEWS', 'NEWPORT NEWS (INDEPENDENT CITY)')\n('NEWPORT NEWS', 'NEWPORT NEWS (INDEPENDENT CITY)')\n('ST CLAIR',  'EAU CLAIRE')\n('ST JOHNS',  'ST. JOHNS')\n('ST JOSEPH',  'ST. JOSEPH')\n('ST LAWRENCE',  'ST. LAWRENCE')\n('ST LOUIS',  'ST. LOUIS')\n('ST LOUIS C

Fix inconsistencies

In [ ]:
county_pairs = [('ALBEMARLE', 'ALBEMARLE + CHARLOTTESVILLE'),
               ('ALEUTIAN ISLANDS', 'ALEUTIANS EAST BOROUGH'),
               ('ANCHORAGE', 'ANCHORAGE MUNICIPALITY'),
               ('BETHEL', 'BETHEL CENSUS AREA'),
               ('BRISTOL BAY', 'BRISTOL BAY BOROUGH'),
               ('CORDOVA-MCCARTHY', 'VALDEZ-CORDOVA CENSUS AREA'),
               ('FAIRBANKS', 'FAIRBANKS NORTH STAR BOROUGH'),
               ('FAIRBANKS NORTH STAR', 'FAIRBANKS NORTH STAR BOROUGH'),
               ('KENAI-COOK INLET', 'KENAI PENINSULA BOROUGH'),
               ('KETCHIKAN', 'KETCHIKAN GATEWAY BOROUGH'),
               ('KETCHIKAN GATEWAY', 'KETCHIKAN GATEWAY BOROUGH'),
               ('KODIAK', 'KODIAK ISLAND BOROUGH'),
               ('KODIAK ISLAND', 'KODIAK ISLAND BOROUGH'),
               ('MAUI', 'MAUI + KALAWAO'),
               ('MC CRACKEN', 'MCCRACKEN'),
               ('MC KEAN', 'MCKEAN'),
               ('MC LEAN', 'MCLEAN'),
               ('MC LENNAN', 'MCLENNAN'),
               ('NEWPORT NEWS', 'NEWPORT NEWS (INDEPENDENT CITY)'),
               ('NEWPORT NEWS', 'NEWPORT NEWS (INDEPENDENT CITY)'),
               ('ST CLAIR', 'EAU CLAIRE'),
               ('ST JOHNS', 'ST. JOHNS'),
               ('ST JOSEPH', 'ST. JOSEPH'),
               ('ST LAWRENCE', 'ST. LAWRENCE'),
               ('ST LOUIS', 'ST. LOUIS'),
               ('ST LOUIS CITY', 'ST. LOUIS (INDEPENDENT CITY)'),
               ('VALDEZ-CHITNA-WHITIE', 'VALDEZ-CORDOVA CENSUS AREA'),
               ('WRANGELL-PETERSBURG', 'WRANGELL-PETERSBURG CENSUS AREA')]

county_dict = {key: value for key, value in county_pairs}
inversed_county_pairs = [(second, first) for first, second in county_pairs]


#CAGDP1['GeoName'] = CAGDP1['GeoName'].replace(inversed_county_pairs)
CAGDP1[CAGDP1.GeoName == 'WRANGELL-PETERSBURG']

,GeoName,Description,Unit,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,State


Rename columns that refer to the same entity so that they are the same in both datasets

In [ ]:
 CAGDP1 = CAGDP1.rename(columns={'GeoName':'COUNTY_NAM', 'State':'COUNTY_ST'})

Create a dataframe for Real GDP (thousands of chained 2012 dollars)

In [ ]:
year2005 = CAGDP1[CAGDP1.Description == 'Real GDP (thousands of chained 2012 dollars) '][['COUNTY_NAM', 'COUNTY_ST', '2005']]
year2005 = year2005.rename(columns={'2005': 'Value of Real GDP (thousands of chained 2012 dollars)'}) 
year2005['Year'] = [2005 for x in range(year2005.shape[0])]
Real_GDP = pd.merge(incidents_airport_locations, year2005, on=['COUNTY_NAM', 'COUNTY_ST', 'Year'], how='left') 

for year in range(2006,2015):
    yearly_data = CAGDP1[CAGDP1.Description == 'Real GDP (thousands of chained 2012 dollars) '][['COUNTY_NAM', 'COUNTY_ST', str(year)]]
    yearly_data = yearly_data.rename(columns={str(year): 'Value of Real GDP (thousands of chained 2012 dollars)'}) 
    yearly_data['Year'] = [year for x in range(yearly_data.shape[0])]
    Real_GDP = pd.merge(Real_GDP, yearly_data, on=['COUNTY_NAM', 'COUNTY_ST', 'Year'], how='left')
    merged_col = Real_GDP.iloc[:,-2].combine_first(Real_GDP.iloc[:,-1])
    Real_GDP = Real_GDP.drop(Real_GDP.columns[-1], axis=1)
    Real_GDP = Real_GDP.drop(Real_GDP.columns[-1], axis=1)
    Real_GDP['Value of Real GDP (thousands of chained 2012 dollars)'] = merged_col



Create a dataframe for the Current-dollar GDP (thousands of current dollars) 

In [ ]:
year2005 = CAGDP1[CAGDP1.Description == 'Current-dollar GDP (thousands of current dollars) '][['COUNTY_NAM', 'COUNTY_ST', '2005']]
year2005 = year2005.rename(columns={'2005': 'Value of Current-dollar GDP (thousands of current dollars)'}) 
year2005['Year'] = [2005 for x in range(year2005.shape[0])]
Current_GDP = pd.merge(incidents_airport_locations, year2005, on=['COUNTY_NAM', 'COUNTY_ST', 'Year'], how='left') 

for year in range(2006,2015):
    yearly_data = CAGDP1[CAGDP1.Description == 'Current-dollar GDP (thousands of current dollars) '][['COUNTY_NAM', 'COUNTY_ST', str(year)]]
    yearly_data = yearly_data.rename(columns={str(year): 'Value of Current-dollar GDP (thousands of current dollars)'}) 
    yearly_data['Year'] = [year for x in range(yearly_data.shape[0])]
    Current_GDP = pd.merge(Current_GDP, yearly_data, on=['COUNTY_NAM', 'COUNTY_ST', 'Year'], how='left')
    merged_col = Current_GDP.iloc[:,-2].combine_first(Current_GDP.iloc[:,-1])
    Current_GDP = Current_GDP.drop(Current_GDP.columns[-1], axis=1)
    Current_GDP = Current_GDP.drop(Current_GDP.columns[-1], axis=1)
    Current_GDP['Value of Current-dollar GDP (thousands of current dollars)'] = merged_col

Current_GDP

,Claim Number,Date Received,Incident Date,Airport Code,Airport Name,Airline Name,Claim Type,Claim Site,Item,Disposition,Close Amount,Year,STATE_NAME,COUNTY_NAM,COUNTY_ST,LATITUDE,LONGITUDE,Value of Current-dollar GDP (thousands of current dollars)
0,2006010699026,2005-01-02,2005-12-06,ABQ,Albuquerque International Sunport Airport,America West,Property Damage,Checkpoint,"Clothing - Shoes, belts, accessories, etc.",Approved,395,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,28218154.0
1,2005020475693,2005-01-19,2005-01-09,ABQ,Albuquerque International Sunport Airport,Delta Air Lines,Passenger Property Loss,Checked Baggage,Other,Denied,NaN,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,28218154.0
2,2005021075827,2005-01-19,2005-01-09,ABQ,Albuquerque International Sunport Airport,Southwest Airlines,Property Damage,Checked Baggage,Luggage (all types including footlockers),Denied,0,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,28218154.0
3,2005021676222,2005-01-27,2005-01-07,ABQ,Albuquerque International Sunport Airport,Delta Air Lines,Passenger Property Loss,Checked Baggage,Other,Denied,0,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,28218154.0
4,2005030776999,2005-02-01,2005-01-05,ABQ,Albuquerque International Sunport Airport,Southwest Airlines,Passenger Property Loss,Checked Baggage,"Clothing - Shoes, belts, accessories, etc.",Approved,60,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,28218154.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135030,2014120819020,2014-11-26,2014-08-11,SBN,South Bend Regional,UAL,Property Damage,Checked Baggage,Cosmetics & Grooming,-,-,2014,INDIANA,ST JOSEPH,IN,41.708222,-86.317333,NaN
135031,2014121019094,2014-12-10,2014-11-28,ENV,Wendover Municipal Airport,Allegiant Air,Property Damage,Checked Baggage,Baggage/Cases/Purses,-,-,2014,UTAH,TOOELE,UT,40.718694,-114.030889,1787205.0
135032,2014122419398,2014-12-24,2014-12-03,ENV,Wendover Municipal Airport,Allegiant Air,Property Damage,Checked Baggage,Cosmetics & Grooming,-,-,2014,UTAH,TOOELE,UT,40.718694,-114.030889,1787205.0
135033,2014121619235,2014-12-16,2014-09-14,ITH,"Tompkins County, Ithaca",USAir,Property Damage,Checked Baggage,-,-,-,2014,NEW YORK,TOMPKINS,NY,42.491361,-76.458722,5619848.0


Create a dataframe for the Chain-type quantity indexes for real GDP 

In [ ]:
year2005 = CAGDP1[CAGDP1.Description == 'Chain-type quantity indexes for real GDP '][['COUNTY_NAM', 'COUNTY_ST', '2005']]
year2005 = year2005.rename(columns={'2005': 'Value of Chain-type quantity indexes for real GDP'}) 
year2005['Year'] = [2005 for x in range(year2005.shape[0])]
Chain_GDP = pd.merge(incidents_airport_locations, year2005, on=['COUNTY_NAM', 'COUNTY_ST', 'Year'], how='left') 

for year in range(2006,2015):
    yearly_data = CAGDP1[CAGDP1.Description == 'Chain-type quantity indexes for real GDP '][['COUNTY_NAM', 'COUNTY_ST', str(year)]]
    yearly_data = yearly_data.rename(columns={str(year): 'Value of Chain-type quantity indexes for real GDP'}) 
    yearly_data['Year'] = [year for x in range(yearly_data.shape[0])]
    Chain_GDP = pd.merge(Chain_GDP, yearly_data, on=['COUNTY_NAM', 'COUNTY_ST', 'Year'], how='left')
    merged_col = Chain_GDP.iloc[:,-2].combine_first(Chain_GDP.iloc[:,-1])
    Chain_GDP = Chain_GDP.drop(Chain_GDP.columns[-1], axis=1)
    Chain_GDP = Chain_GDP.drop(Chain_GDP.columns[-1], axis=1)
    Chain_GDP['Value of Chain-type quantity indexes for real GDP'] = merged_col

Chain_GDP

,Claim Number,Date Received,Incident Date,Airport Code,Airport Name,Airline Name,Claim Type,Claim Site,Item,Disposition,Close Amount,Year,STATE_NAME,COUNTY_NAM,COUNTY_ST,LATITUDE,LONGITUDE,Value of Chain-type quantity indexes for real GDP
0,2006010699026,2005-01-02,2005-12-06,ABQ,Albuquerque International Sunport Airport,America West,Property Damage,Checkpoint,"Clothing - Shoes, belts, accessories, etc.",Approved,395,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,100.114
1,2005020475693,2005-01-19,2005-01-09,ABQ,Albuquerque International Sunport Airport,Delta Air Lines,Passenger Property Loss,Checked Baggage,Other,Denied,NaN,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,100.114
2,2005021075827,2005-01-19,2005-01-09,ABQ,Albuquerque International Sunport Airport,Southwest Airlines,Property Damage,Checked Baggage,Luggage (all types including footlockers),Denied,0,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,100.114
3,2005021676222,2005-01-27,2005-01-07,ABQ,Albuquerque International Sunport Airport,Delta Air Lines,Passenger Property Loss,Checked Baggage,Other,Denied,0,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,100.114
4,2005030776999,2005-02-01,2005-01-05,ABQ,Albuquerque International Sunport Airport,Southwest Airlines,Passenger Property Loss,Checked Baggage,"Clothing - Shoes, belts, accessories, etc.",Approved,60,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,100.114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135030,2014120819020,2014-11-26,2014-08-11,SBN,South Bend Regional,UAL,Property Damage,Checked Baggage,Cosmetics & Grooming,-,-,2014,INDIANA,ST JOSEPH,IN,41.708222,-86.317333,NaN
135031,2014121019094,2014-12-10,2014-11-28,ENV,Wendover Municipal Airport,Allegiant Air,Property Damage,Checked Baggage,Baggage/Cases/Purses,-,-,2014,UTAH,TOOELE,UT,40.718694,-114.030889,91.201
135032,2014122419398,2014-12-24,2014-12-03,ENV,Wendover Municipal Airport,Allegiant Air,Property Damage,Checked Baggage,Cosmetics & Grooming,-,-,2014,UTAH,TOOELE,UT,40.718694,-114.030889,91.201
135033,2014121619235,2014-12-16,2014-09-14,ITH,"Tompkins County, Ithaca",USAir,Property Damage,Checked Baggage,-,-,-,2014,NEW YORK,TOMPKINS,NY,42.491361,-76.458722,96.351


Merge the three dataframes

In [ ]:
GDP = Real_GDP
GDP['Value of Current-dollar GDP (thousands of current dollars)'] = Current_GDP['Value of Current-dollar GDP (thousands of current dollars)']
GDP['Value of Chain-type quantity indexes for real GDP'] = Chain_GDP['Value of Chain-type quantity indexes for real GDP']

In [ ]:
GDP[GDP.Year == 2014]

,Claim Number,Date Received,Incident Date,Airport Code,Airport Name,Airline Name,Claim Type,Claim Site,Item,Disposition,Close Amount,Year,STATE_NAME,COUNTY_NAM,COUNTY_ST,LATITUDE,LONGITUDE,Value of Real GDP (thousands of chained 2012 dollars),Value of Current-dollar GDP (thousands of current dollars),Value of Chain-type quantity indexes for real GDP
127382,2014080215586,2014-07-17,2014-06-30,MCO,Orlando International Airport,Delta Air Lines,Property Damage,Checked Baggage,-,Deny,0,2014,FLORIDA,ORANGE,FL,28.429389,-81.309000,74601727.0,77556519.0,106.901
127383,2014043013619,2014-04-25,2014-04-11,MCO,Orlando International Airport,Virgin Atlantic,Property Damage,Checked Baggage,Baggage/Cases/Purses,Approve in Full,85,2014,FLORIDA,ORANGE,FL,28.429389,-81.309000,74601727.0,77556519.0,106.901
127384,2014080815786,2014-07-21,2014-07-12,MCO,Orlando International Airport,USAir,Property Damage,Checked Baggage,Baggage/Cases/Purses,Deny,0,2014,FLORIDA,ORANGE,FL,28.429389,-81.309000,74601727.0,77556519.0,106.901
127385,2014100617397,2014-09-25,2014-08-29,MCO,Orlando International Airport,Southwest Airlines,Passenger Property Loss,Checked Baggage,Clothing,-,-,2014,FLORIDA,ORANGE,FL,28.429389,-81.309000,74601727.0,77556519.0,106.901
127386,2014010810649,2014-01-06,2014-01-04,MCO,Orlando International Airport,Southwest Airlines,Property Damage,Checkpoint,Personal Electronics,Deny,0,2014,FLORIDA,ORANGE,FL,28.429389,-81.309000,74601727.0,77556519.0,106.901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135030,2014120819020,2014-11-26,2014-08-11,SBN,South Bend Regional,UAL,Property Damage,Checked Baggage,Cosmetics & Grooming,-,-,2014,INDIANA,ST JOSEPH,IN,41.708222,-86.317333,NaN,NaN,NaN
135031,2014121019094,2014-12-10,2014-11-28,ENV,Wendover Municipal Airport,Allegiant Air,Property Damage,Checked Baggage,Baggage/Cases/Purses,-,-,2014,UTAH,TOOELE,UT,40.718694,-114.030889,1751071.0,1787205.0,91.201
135032,2014122419398,2014-12-24,2014-12-03,ENV,Wendover Municipal Airport,Allegiant Air,Property Damage,Checked Baggage,Cosmetics & Grooming,-,-,2014,UTAH,TOOELE,UT,40.718694,-114.030889,1751071.0,1787205.0,91.201
135033,2014121619235,2014-12-16,2014-09-14,ITH,"Tompkins County, Ithaca",USAir,Property Damage,Checked Baggage,-,-,-,2014,NEW YORK,TOMPKINS,NY,42.491361,-76.458722,5353522.0,5619848.0,96.351


EXTRACT "SAINC" is an acronym for the State Annual Personal Income and Employment series,

Air transportation Support activities for transportation

In [ ]:
with zipfile.ZipFile("/content/SAINC.zip", "r") as zip_ref:
    zip_ref.extractall("SAINC")

In [ ]:
import chardet

SAEMP = pd.DataFrame()
count = 0

for filename in os.listdir('/content/SAINC'):

    state = filename.split('_')[1]
    if len(state) ==2 :
        
        if filename.split('_')[0] == 'SAEMP25N' and filename.split('_')[-1] == '2021.csv' :
            count +=1
            with open(os.path.join('/content/SAINC', filename), 'rb') as f:
                result = chardet.detect(f.read())
            print(state, result['encoding'])
            jobs = pd.read_csv(os.path.join('/content/SAINC', filename), encoding=result['encoding']) #, on_bad_lines='skip'
            jobs = jobs[['GeoName', 'Description','Unit','2005','2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014']]
            jobs['State'] = [state for x in range(jobs.shape[0])]
            SAEMP = pd.concat([SAEMP, jobs], ignore_index=True)

#print(set(SAINC.Description.values))

SAEMP[(SAEMP.Description == '    Air transportation ') | (SAEMP.Description == '    Support activities for transportation ') | (SAEMP.Description ==  ' Wage and salary employment ')]
#SAEMP['GeoName'] = SAEMP['GeoName'].replace(inversed_county_pairs)
SAEMP = SAEMP.rename(columns={ 'State':'COUNTY_ST'})
#'    Air transportation ': 'Air transportation', '    Support activities for transportation ':'Support activities for transportation', ' Wage and salary employment ':'Wage and salary employment'


US ascii
TX ascii
AZ ascii
OH ascii
NJ ascii
IN ascii
DC ascii
CT ascii
NY ascii
WA ascii
TN ascii
RI ascii
SC ascii
FL ascii
SD ascii
OK ascii
NV ascii
MT ascii
MS ascii
HI ascii
NM ascii
VT ascii
KY ascii
NH ascii
CO ascii
IL ascii
ND ascii
NE ascii


<ipython-input-48-19e002f4d0ad>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs['State'] = [state for x in range(jobs.shape[0])]


CA ascii
AR ascii
MO ascii
MI ascii
OR ascii
VA ascii
WI ascii
WV ascii
KS ascii
PA ascii
MD ascii
UT ascii
DE ascii
LA ascii
GA ascii
MA ascii
IA ascii
AL ascii
WY ascii
MN ascii
ID ascii
NC ascii
AK ascii
ME ascii


In [ ]:
year2005 = SAEMP[SAEMP.Description == '    Air transportation '][['COUNTY_ST', '2005']]
year2005 = year2005.rename(columns={'2005': 'Number of Air transportation jobs'}) 
year2005['Year'] = [2005 for x in range(year2005.shape[0])]
AIR_TRANSP = pd.merge(GDP, year2005, on=['COUNTY_ST', 'Year'], how='left') 

for year in range(2006,2015):
    yearly_data = SAEMP[SAEMP.Description == '    Air transportation '][[ 'COUNTY_ST', str(year)]]
    yearly_data = yearly_data.rename(columns={str(year): 'Number of Air transportation jobs'}) 
    yearly_data['Year'] = [year for x in range(yearly_data.shape[0])]
    AIR_TRANSP = pd.merge(AIR_TRANSP, yearly_data, on=[ 'COUNTY_ST', 'Year'], how='left')
    merged_col = AIR_TRANSP.iloc[:,-2].combine_first(AIR_TRANSP.iloc[:,-1])
    AIR_TRANSP = AIR_TRANSP.drop(AIR_TRANSP.columns[-1], axis=1)
    AIR_TRANSP = AIR_TRANSP.drop(AIR_TRANSP.columns[-1], axis=1)
    AIR_TRANSP['Number of air transportation jobs of state'] = merged_col

AIR_TRANSP

,Claim Number,Date Received,Incident Date,Airport Code,Airport Name,Airline Name,Claim Type,Claim Site,Item,Disposition,...,Year,STATE_NAME,COUNTY_NAM,COUNTY_ST,LATITUDE,LONGITUDE,Value of Real GDP (thousands of chained 2012 dollars),Value of Current-dollar GDP (thousands of current dollars),Value of Chain-type quantity indexes for real GDP,Number of air transportation jobs of state
0,2006010699026,2005-01-02,2005-12-06,ABQ,Albuquerque International Sunport Airport,America West,Property Damage,Checkpoint,"Clothing - Shoes, belts, accessories, etc.",Approved,...,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,32151536.0,28218154.0,100.114,1773
1,2005020475693,2005-01-19,2005-01-09,ABQ,Albuquerque International Sunport Airport,Delta Air Lines,Passenger Property Loss,Checked Baggage,Other,Denied,...,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,32151536.0,28218154.0,100.114,1773
2,2005021075827,2005-01-19,2005-01-09,ABQ,Albuquerque International Sunport Airport,Southwest Airlines,Property Damage,Checked Baggage,Luggage (all types including footlockers),Denied,...,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,32151536.0,28218154.0,100.114,1773
3,2005021676222,2005-01-27,2005-01-07,ABQ,Albuquerque International Sunport Airport,Delta Air Lines,Passenger Property Loss,Checked Baggage,Other,Denied,...,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,32151536.0,28218154.0,100.114,1773
4,2005030776999,2005-02-01,2005-01-05,ABQ,Albuquerque International Sunport Airport,Southwest Airlines,Passenger Property Loss,Checked Baggage,"Clothing - Shoes, belts, accessories, etc.",Approved,...,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,32151536.0,28218154.0,100.114,1773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135030,2014120819020,2014-11-26,2014-08-11,SBN,South Bend Regional,UAL,Property Damage,Checked Baggage,Cosmetics & Grooming,-,...,2014,INDIANA,ST JOSEPH,IN,41.708222,-86.317333,NaN,NaN,NaN,7100
135031,2014121019094,2014-12-10,2014-11-28,ENV,Wendover Municipal Airport,Allegiant Air,Property Damage,Checked Baggage,Baggage/Cases/Purses,-,...,2014,UTAH,TOOELE,UT,40.718694,-114.030889,1751071.0,1787205.0,91.201,6530
135032,2014122419398,2014-12-24,2014-12-03,ENV,Wendover Municipal Airport,Allegiant Air,Property Damage,Checked Baggage,Cosmetics & Grooming,-,...,2014,UTAH,TOOELE,UT,40.718694,-114.030889,1751071.0,1787205.0,91.201,6530
135033,2014121619235,2014-12-16,2014-09-14,ITH,"Tompkins County, Ithaca",USAir,Property Damage,Checked Baggage,-,-,...,2014,NEW YORK,TOMPKINS,NY,42.491361,-76.458722,5353522.0,5619848.0,96.351,32804


In [ ]:
year2005 = SAEMP[SAEMP.Description == '    Support activities for transportation '][['COUNTY_ST', '2005']]
year2005 = year2005.rename(columns={'2005': 'Number of support activities for transportation jobs'}) 
year2005['Year'] = [2005 for x in range(year2005.shape[0])]
SUPPORT_TRANSP = pd.merge(GDP, year2005, on=['COUNTY_ST', 'Year'], how='left') 

for year in range(2006,2015):
    yearly_data = SAEMP[SAEMP.Description == '    Support activities for transportation '][[ 'COUNTY_ST', str(year)]]
    yearly_data = yearly_data.rename(columns={str(year): 'Number of Air transportation jobs'}) 
    yearly_data['Year'] = [year for x in range(yearly_data.shape[0])]
    SUPPORT_TRANSP = pd.merge(SUPPORT_TRANSP, yearly_data, on=[ 'COUNTY_ST', 'Year'], how='left')
    merged_col = SUPPORT_TRANSP.iloc[:,-2].combine_first(SUPPORT_TRANSP.iloc[:,-1])
    SUPPORT_TRANSP = SUPPORT_TRANSP.drop(SUPPORT_TRANSP.columns[-1], axis=1)
    SUPPORT_TRANSP = SUPPORT_TRANSP.drop(SUPPORT_TRANSP.columns[-1], axis=1)
    SUPPORT_TRANSP['Number of support activities for transportation jobs of state'] = merged_col

SUPPORT_TRANSP



,Claim Number,Date Received,Incident Date,Airport Code,Airport Name,Airline Name,Claim Type,Claim Site,Item,Disposition,...,Year,STATE_NAME,COUNTY_NAM,COUNTY_ST,LATITUDE,LONGITUDE,Value of Real GDP (thousands of chained 2012 dollars),Value of Current-dollar GDP (thousands of current dollars),Value of Chain-type quantity indexes for real GDP,Number of support activities for transportation jobs of state
0,2006010699026,2005-01-02,2005-12-06,ABQ,Albuquerque International Sunport Airport,America West,Property Damage,Checkpoint,"Clothing - Shoes, belts, accessories, etc.",Approved,...,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,32151536.0,28218154.0,100.114,2293
1,2005020475693,2005-01-19,2005-01-09,ABQ,Albuquerque International Sunport Airport,Delta Air Lines,Passenger Property Loss,Checked Baggage,Other,Denied,...,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,32151536.0,28218154.0,100.114,2293
2,2005021075827,2005-01-19,2005-01-09,ABQ,Albuquerque International Sunport Airport,Southwest Airlines,Property Damage,Checked Baggage,Luggage (all types including footlockers),Denied,...,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,32151536.0,28218154.0,100.114,2293
3,2005021676222,2005-01-27,2005-01-07,ABQ,Albuquerque International Sunport Airport,Delta Air Lines,Passenger Property Loss,Checked Baggage,Other,Denied,...,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,32151536.0,28218154.0,100.114,2293
4,2005030776999,2005-02-01,2005-01-05,ABQ,Albuquerque International Sunport Airport,Southwest Airlines,Passenger Property Loss,Checked Baggage,"Clothing - Shoes, belts, accessories, etc.",Approved,...,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,32151536.0,28218154.0,100.114,2293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135030,2014120819020,2014-11-26,2014-08-11,SBN,South Bend Regional,UAL,Property Damage,Checked Baggage,Cosmetics & Grooming,-,...,2014,INDIANA,ST JOSEPH,IN,41.708222,-86.317333,NaN,NaN,NaN,10572
135031,2014121019094,2014-12-10,2014-11-28,ENV,Wendover Municipal Airport,Allegiant Air,Property Damage,Checked Baggage,Baggage/Cases/Purses,-,...,2014,UTAH,TOOELE,UT,40.718694,-114.030889,1751071.0,1787205.0,91.201,4910
135032,2014122419398,2014-12-24,2014-12-03,ENV,Wendover Municipal Airport,Allegiant Air,Property Damage,Checked Baggage,Cosmetics & Grooming,-,...,2014,UTAH,TOOELE,UT,40.718694,-114.030889,1751071.0,1787205.0,91.201,4910
135033,2014121619235,2014-12-16,2014-09-14,ITH,"Tompkins County, Ithaca",USAir,Property Damage,Checked Baggage,-,-,...,2014,NEW YORK,TOMPKINS,NY,42.491361,-76.458722,5353522.0,5619848.0,96.351,30575


In [ ]:
year2005 = SAEMP[SAEMP.Description == ' Wage and salary employment '][['COUNTY_ST', '2005']]
year2005 = year2005.rename(columns={'2005': 'Number of wage and salary employment positions of state'}) 
year2005['Year'] = [2005 for x in range(year2005.shape[0])]
WAGE_SALARY = pd.merge(GDP, year2005, on=['COUNTY_ST', 'Year'], how='left') 

for year in range(2006,2015):
    yearly_data = SAEMP[SAEMP.Description == ' Wage and salary employment '][[ 'COUNTY_ST', str(year)]]
    yearly_data = yearly_data.rename(columns={str(year): 'Number of wage and salary employment positions of state'}) 
    yearly_data['Year'] = [year for x in range(yearly_data.shape[0])]
    WAGE_SALARY = pd.merge(WAGE_SALARY, yearly_data, on=[ 'COUNTY_ST', 'Year'], how='left')
    merged_col = WAGE_SALARY.iloc[:,-2].combine_first(WAGE_SALARY.iloc[:,-1])
    WAGE_SALARY = WAGE_SALARY.drop(WAGE_SALARY.columns[-1], axis=1)
    WAGE_SALARY = WAGE_SALARY.drop(WAGE_SALARY.columns[-1], axis=1)
    WAGE_SALARY['Number of wage and salary employment positions of state'] = merged_col

WAGE_SALARY



,Claim Number,Date Received,Incident Date,Airport Code,Airport Name,Airline Name,Claim Type,Claim Site,Item,Disposition,...,Year,STATE_NAME,COUNTY_NAM,COUNTY_ST,LATITUDE,LONGITUDE,Value of Real GDP (thousands of chained 2012 dollars),Value of Current-dollar GDP (thousands of current dollars),Value of Chain-type quantity indexes for real GDP,Number of wage and salary employment positions of state
0,2006010699026,2005-01-02,2005-12-06,ABQ,Albuquerque International Sunport Airport,America West,Property Damage,Checkpoint,"Clothing - Shoes, belts, accessories, etc.",Approved,...,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,32151536.0,28218154.0,100.114,844235
1,2005020475693,2005-01-19,2005-01-09,ABQ,Albuquerque International Sunport Airport,Delta Air Lines,Passenger Property Loss,Checked Baggage,Other,Denied,...,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,32151536.0,28218154.0,100.114,844235
2,2005021075827,2005-01-19,2005-01-09,ABQ,Albuquerque International Sunport Airport,Southwest Airlines,Property Damage,Checked Baggage,Luggage (all types including footlockers),Denied,...,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,32151536.0,28218154.0,100.114,844235
3,2005021676222,2005-01-27,2005-01-07,ABQ,Albuquerque International Sunport Airport,Delta Air Lines,Passenger Property Loss,Checked Baggage,Other,Denied,...,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,32151536.0,28218154.0,100.114,844235
4,2005030776999,2005-02-01,2005-01-05,ABQ,Albuquerque International Sunport Airport,Southwest Airlines,Passenger Property Loss,Checked Baggage,"Clothing - Shoes, belts, accessories, etc.",Approved,...,2005,NEW MEXICO,BERNALILLO,NM,35.040222,-106.609194,32151536.0,28218154.0,100.114,844235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135030,2014120819020,2014-11-26,2014-08-11,SBN,South Bend Regional,UAL,Property Damage,Checked Baggage,Cosmetics & Grooming,-,...,2014,INDIANA,ST JOSEPH,IN,41.708222,-86.317333,NaN,NaN,NaN,3054029
135031,2014121019094,2014-12-10,2014-11-28,ENV,Wendover Municipal Airport,Allegiant Air,Property Damage,Checked Baggage,Baggage/Cases/Purses,-,...,2014,UTAH,TOOELE,UT,40.718694,-114.030889,1751071.0,1787205.0,91.201,1381259
135032,2014122419398,2014-12-24,2014-12-03,ENV,Wendover Municipal Airport,Allegiant Air,Property Damage,Checked Baggage,Cosmetics & Grooming,-,...,2014,UTAH,TOOELE,UT,40.718694,-114.030889,1751071.0,1787205.0,91.201,1381259
135033,2014121619235,2014-12-16,2014-09-14,ITH,"Tompkins County, Ithaca",USAir,Property Damage,Checked Baggage,-,-,...,2014,NEW YORK,TOMPKINS,NY,42.491361,-76.458722,5353522.0,5619848.0,96.351,9220265


In [ ]:
GDP_JOBS = AIR_TRANSP
GDP_JOBS['Number of support activities for transportation jobs of state'] = SUPPORT_TRANSP['Number of support activities for transportation jobs of state'] 
GDP_JOBS['Number of wage and salary employment positions of state'] = WAGE_SALARY['Number of wage and salary employment positions of state']

In [ ]:
GDP_JOBS

,Claim Number,Date Received,Incident Date,Airport Code,Airport Name,Airline Name,Claim Type,Claim Site,Item,Disposition,...,COUNTY_NAM,COUNTY_ST,LATITUDE,LONGITUDE,Value of Real GDP (thousands of chained 2012 dollars),Value of Current-dollar GDP (thousands of current dollars),Value of Chain-type quantity indexes for real GDP,Number of air transportation jobs of state,Number of support activities for transportation jobs of state,Number of wage and salary employment positions of state
0,2006010699026,2005-01-02,2005-12-06,ABQ,Albuquerque International Sunport Airport,America West,Property Damage,Checkpoint,"Clothing - Shoes, belts, accessories, etc.",Approved,...,BERNALILLO,NM,35.040222,-106.609194,32151536.0,28218154.0,100.114,1773,2293,844235
1,2005020475693,2005-01-19,2005-01-09,ABQ,Albuquerque International Sunport Airport,Delta Air Lines,Passenger Property Loss,Checked Baggage,Other,Denied,...,BERNALILLO,NM,35.040222,-106.609194,32151536.0,28218154.0,100.114,1773,2293,844235
2,2005021075827,2005-01-19,2005-01-09,ABQ,Albuquerque International Sunport Airport,Southwest Airlines,Property Damage,Checked Baggage,Luggage (all types including footlockers),Denied,...,BERNALILLO,NM,35.040222,-106.609194,32151536.0,28218154.0,100.114,1773,2293,844235
3,2005021676222,2005-01-27,2005-01-07,ABQ,Albuquerque International Sunport Airport,Delta Air Lines,Passenger Property Loss,Checked Baggage,Other,Denied,...,BERNALILLO,NM,35.040222,-106.609194,32151536.0,28218154.0,100.114,1773,2293,844235
4,2005030776999,2005-02-01,2005-01-05,ABQ,Albuquerque International Sunport Airport,Southwest Airlines,Passenger Property Loss,Checked Baggage,"Clothing - Shoes, belts, accessories, etc.",Approved,...,BERNALILLO,NM,35.040222,-106.609194,32151536.0,28218154.0,100.114,1773,2293,844235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135030,2014120819020,2014-11-26,2014-08-11,SBN,South Bend Regional,UAL,Property Damage,Checked Baggage,Cosmetics & Grooming,-,...,ST JOSEPH,IN,41.708222,-86.317333,NaN,NaN,NaN,7100,10572,3054029
135031,2014121019094,2014-12-10,2014-11-28,ENV,Wendover Municipal Airport,Allegiant Air,Property Damage,Checked Baggage,Baggage/Cases/Purses,-,...,TOOELE,UT,40.718694,-114.030889,1751071.0,1787205.0,91.201,6530,4910,1381259
135032,2014122419398,2014-12-24,2014-12-03,ENV,Wendover Municipal Airport,Allegiant Air,Property Damage,Checked Baggage,Cosmetics & Grooming,-,...,TOOELE,UT,40.718694,-114.030889,1751071.0,1787205.0,91.201,6530,4910,1381259
135033,2014121619235,2014-12-16,2014-09-14,ITH,"Tompkins County, Ithaca",USAir,Property Damage,Checked Baggage,-,-,...,TOMPKINS,NY,42.491361,-76.458722,5353522.0,5619848.0,96.351,32804,30575,9220265


In [ ]:
GDP_JOBS.to_excel('augmented_dataset.xlsx')